In [11]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('data.csv')

In [13]:
df.head(5)

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             116 non-null    int64  
 1   BMI             116 non-null    float64
 2   Glucose         116 non-null    int64  
 3   Insulin         116 non-null    float64
 4   HOMA            116 non-null    float64
 5   Leptin          116 non-null    float64
 6   Adiponectin     116 non-null    float64
 7   Resistin        116 non-null    float64
 8   MCP.1           116 non-null    float64
 9   Classification  116 non-null    int64  
dtypes: float64(7), int64(3)
memory usage: 9.2 KB


In [30]:
print(sum(df['Age']) / len(df['Age']))

57.30172413793103


In [25]:
def gini(y):
    # Calculate the Gini impurity of a target variable
    classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    gini_impurity = 1 - np.sum(probabilities ** 2)
    return gini_impurity

In [28]:
y = df['Classification'].values.reshape(-1, 1)
X = df.drop(columns='Classification')
print(X.shape)
print(y.shape)

(116, 9)
(116, 1)


In [40]:
a = gini(y)

In [43]:
def gini_index(X, y, feature_idx, threshold):
    # Calculate the Gini index for a given feature and threshold
    #gini_parent = gini(y)

    # Split the data based on the given feature and threshold - relabel true-false
    left_mask = X.iloc[:, feature_idx] <= threshold
    right_mask = X.iloc[:, feature_idx] > threshold
    y_left = y[left_mask]
    y_right = y[right_mask]

    # Calculate the Gini impurity of the left and right subsets
    gini_left = gini(y_left)
    gini_right = gini(y_right)

    # Calculate the Gini index
    n_left = len(y_left)
    n_right = len(y_right)
    n_total = len(y)
    gini_index = (n_left / n_total) * gini_left + (n_right / n_total) * gini_right
    return gini_index

In [35]:
print(X.iloc[:, 1])

0      23.500000
1      20.690495
2      23.124670
3      21.367521
4      21.111111
         ...    
111    26.850000
112    26.840000
113    32.050000
114    25.590000
115    27.180000
Name: BMI, Length: 116, dtype: float64


In [46]:
b = gini_index(X, y, 5, 57.3)

In [47]:
a - b

0.0003963535473643365

In [57]:
left_mask = X.iloc[:, 0] <= 57.3
right_mask = X.iloc[:, 0] > 57.3
x_left = X[left_mask]
print(x_left)

     Age        BMI  Glucose  Insulin       HOMA   Leptin  Adiponectin  \
0     48  23.500000       70    2.707   0.467409   8.8071     9.702400   
5     49  22.854458       92    3.226   0.732087   6.8317    13.679750   
10    34  21.470000       78    3.469   0.667436  14.5700    13.110000   
11    29  23.010000       82    5.663   1.145436  35.5900    26.720000   
12    25  22.860000       82    4.090   0.827271  20.4500    23.670000   
13    24  18.670000       88    6.107   1.330000   8.8800    36.060000   
14    38  23.340000       75    5.782   1.069670  15.2600    17.950000   
15    44  20.760000       86    7.553   1.600000  14.0900    20.320000   
16    47  22.030000       84    2.869   0.590000  26.6500    38.040000   
19    32  36.512637       87   14.026   3.009980  49.3727     5.100000   
20    36  28.576676       86    4.345   0.921719  15.1248     8.600000   
21    34  31.975015       87    4.530   0.972138  28.7502     7.642760   
22    29  32.270788       84    5.810 

In [52]:
print(np.unique(left_mask, return_counts=True))

(array([False,  True]), array([57, 59], dtype=int64))


In [ ]:
def find_best_split(self, X, y):
    best_gini_index = np.inf
    best_feature = None
    best_threshold = None

    for feature_idx in range(X.shape[1]):
        thresholds = np.unique(X[:, feature_idx])
        for threshold in thresholds:
            gini_index = self.gini_index(X, y, feature_idx, threshold)
            if gini_index < best_gini_index:
                best_gini_index = gini_index
                best_feature = feature_idx
                best_threshold = threshold

    return best_feature, best_threshold

In [56]:
thresholds = np.unique(X.iloc[:, 0])
print(thresholds)


[24 25 28 29 32 34 35 36 38 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54
 55 57 58 59 60 61 62 64 65 66 67 68 69 71 72 73 74 75 76 77 78 81 82 83
 85 86 89]
0      48
1      83
2      82
3      68
4      86
       ..
111    45
112    62
113    65
114    72
115    86
Name: Age, Length: 116, dtype: int64


In [ ]:
def build_tree(self, X, y, used_features=None):
    if used_features is None:
        used_features = set()

    # Base case: If all samples belong to the same class, return a leaf node
    if len(np.unique(y)) == 1:
        return {'class': y[0]}

    # Base case: If we have no more features to split on, return the majority class
    if X.shape[1] == 0 or len(used_features) == X.shape[1]:
        unique_classes, counts = np.unique(y, return_counts=True)
        majority_class = unique_classes[np.argmax(counts)]
        return {'class': majority_class}

    # Find the best feature and threshold to split the data, excluding the used features
    best_feature, best_threshold = self.find_best_split(X, y, used_features)

    # ...

    # Recursive call to build subtrees, passing the updated used_features set
    left_subtree = self.build_tree(X_left, y_left, used_features | {best_feature})
    right_subtree = self.build_tree(X_right, y_right, used_features | {best_feature})

    # ...

    return node